# 손글씨, 와인, 유방암 판별기 만들기



이번에는 scikit-learn이라는 라이브러리를 이용해 판별기를 만들어 보겠습니다. 그 전에 이번 판별기를 만들때 필요한 두가지 패키지에 대해 알아보도록 하겠습니다.


## 1.라이브러리

### (1) scikit-learn

![scikit-learn cheatsheet](./images/scikit-learn-cheatsheet.png)

scikit-learn은 NumPy, SciPy 그리고 matplotlib로 만들어진 phython 라이브러리이며 기본적인 머신러닝 기능들을 지원하여 **지도** 및 **비지도 학습**을 쉽게 사용하도록 만들어주는 라이브러리인데요.

쉽게 연습할 수 있게 기본적인 데이터들도 제공해주는데 **Toy datasets**라는 작은 데이터셋과 **Real world datasets**라는 큰 데이터셋을 제공해줍니다.

그럼 scikit-learn의 몇가지 기능을 활용해보겠습니다.

In [54]:
from sklearn.datasets import load_iris

# scikit-learn에서 제공해주는 toy datasets중 iris datasets를 가져오는 함수
iris = load_iris()
print(type(dir(iris)))
print(iris.keys())

# iris key값들을 이용해 iris datasets 확인해보기
iris_data = iris.data
iris_label = iris.target
print(iris_data.shape)
print(iris_label.shape)
print(iris.target_names)
print(iris.DESCR)
print(iris.feature_names)
print(iris.filename)


<class 'list'>
dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])
(150, 4)
(150,)
['setosa' 'versicolor' 'virginica']
.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length: 

### (2) pandas

Python Data Analysis Library(pandas)는 python에서 배열과 시계열들을 다루기 위해서 사용되는 라이브러리인뎅요.

1차원 구조인 **series**와 2차원 구조인 **DataFrame**을 통해 데이터를 처리합니다.

그럼 pandas를 통해 iris dataset을 가공해보겠습니다.

In [55]:
import pandas as pd

# 자료형 변환 (iris데이터, 컬럼명은 특징들로 설정)
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)
# 생성된 배열에 label이라는 class분류 추가
iris_df["label"] = iris.target
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


## 2.데이터 준비하기

이제 데이터를 준비할 차례입니다. 이번에는 scikit-learn에서 제공해주는 dataset들을 사용하기 때문에 바로 데이터 분리부터 시작해보겠습니다.

일단 데이터는 두종류의 데이터들이 필요합니다.

1. .data를 사용해서 가져온 **feature**라는 데이터입니다. 이 데이터들을 입력하여 label을 나눌 것이기 때문에 변수 **x**로 사용하겠습니다.
2. .target을 사용해서 가져온 **label** 혹은 **target**이라는 데이터입니다. 이 데이터들이 찾고자하는 목표이기 때문에 변수 **y**로 사용하겠습니다.

이렇게 두종류의 데이터를 나눈 이후 train set과 test set으로 나누어야 하는데요. 이를 위해서 scikit-learn이 제공하는 train_test_split이라는 함수를 사용하겠습니다.

In [56]:
from sklearn.model_selection import train_test_split

# data와 label을 파라미터로 받고 test_size함수로 train:test 비율설정, 무작위 데이터를 뽑기위한 random_state 설정을 해준다.
X_train, X_test, y_train, y_test = train_test_split(iris_data, 
                                                    iris_label, 
                                                    test_size=0.2, 
                                                    random_state=7)

print('X_train 개수: ', len(X_train), ', X_test 개수: ', len(X_test))
X_test.shape, y_test.shape
X_train.shape, y_train.shape

X_train 개수:  120 , X_test 개수:  30


((120, 4), (120,))

## 3. 모델 학습시키기

머신러닝 모델에는 매우 다양한 종류가 있는데요. 그중 몇가지 기본적인 모델들의 특징들을 가볍게 살펴보겠습니다.

### (1) Decision Tree
의사결정나무라고 불리는 모델입니다. 어떠한 명제가 주어지면 참, 거짓으로 나누어져 **분기**하는데 이 모양이 나무가 뒤집힌 모양같다고 해서 붙여진 이름인데요. 의사결정나무는 비교를 위해 입력 변수 영역을 둘로 나누는 **재귀적분기(*recursive partitioning*)**과 성능 향상을 위해 불필요하게 자세한 영역을 통합하는 **가지치기(*pruning*)**를 통해 학습합니다.

### (2) Random Forest
Decision Tree는 계산복잡도에 비해 좋은 성능과 변수단위로 설명이 가능하다는 장점이 있지만 **특정 데이터**에만 잘 작동한다는 한계가 있었습니다. 그 문제를 극복하기 위해 Decision Tree를 **앙상블 기법(*Ensemble*)** 통해 만든 것이 Random Forest인데요. 각 요소를 무작위적으로 배치하고 각 트릐들끼리 투표를 통해 최종적으로 의사결정을 내리게 됩니다.

### (3) Support Vector Machine(SVM)
SVM은 데이터를 비선형 맵핑을 하여 고차원으로 변환하는데, 이 고차원 데이터에서 최적의 **선형분리**를 찾아 의사결정역역(*Decision Boundary*)를 찾는 것이 SVM의 목표입니다. 이 선형분리를 하는 초평면을 **MMH(*Maximum Marginal Hyperplane*)**라고 하는데, 이 초평면과 맞닿아 있는**(초평면을 찾는데 도움을 주는)** 투플들이 바로 **Support Vector**이라 불립니다. 이렇게 엄격하게 구분을 찾는 방법을 하드마진(*Hard Margin*) 방법이라고 하는데, 이 방법은 현실세계에서는 초평면을 구하기 힘들기 때문에 소프트 마진 방식이 개발되었습니다.

### (4) Logistic Regression
logistic regression은 **회귀**를 사용하여 데이터가 어떠한 클래스에 속할 확률을 **0 ~ 1**사이의 숫자로 나타내고 더 높은 확률의 클래스로 분류해주는 모델인데요. 이런 식으로 데이터가 2개의 클래스 중 어디에 속하는지 분류하는 것을 **2진 분류(*binary classification*)**라고 합니다. 이렇게 데이터를 분류해놓으면 오차가 생기겠죠. 이러한 오차를 표현하는 방법을 **오차행렬(*confusion matrix*)**라고 합니다. 이는 각각 **True Positive**(양성으로 판단한 양성), **False Negative**(음성으로 판단한 음성), **False Positive**(양성이라고 판단한 음성), **True Negative**(음성이라고 판단한 양성)으로 분류되어 있습니다.  
  
![confusion matrix](./images/confusionMatrix.jpg)  
  
위의 이미지를 보시면 4종류로 분류된 결과값들로 성능을 계산한 값들이 있습니다. 그중 **Recall(Sensitivity)**과 **Precision**에 대해 조금 설명해보겠습니다.(**Accuracy**는 모델의 성능 자체이니 당연히 중요한 값이겠죠)
Recall은 FN의 값이 커질수록 오차율이 늘어납니다. 양성을 음성으로 잘못 판단한 경우인데요, 병원에서 암진단을 이런 식으로 하면 큰일이 나겠죠? 스팸메일 분류기는 음성(정상 메일)을 양성(스팸)으로 판단해 스팸함에 넣어버리는 것이 반대의 경우보다 더 큰 문제가 되겠네요.


**\* Stochastic Gradient Descent(SGD)** : 앞에서 소개한 4가지 모델 이외에도 SGD classifier를 하나 더 사용해볼 건데요. SGD는 엄밀히 따지면 모델이 아닌 **최적화 기술**입니다. 다만 scikit-learn API는 SGD가 적용된 SVM과 LogisticRegression을 기본적으로 제공해주기 때문에 예시로 사용해보겠습니다.

## 4. 분류기 만들어보기

### (1) 손글씨 분류기

scikit-learn에서 제공하는 load_digits 메서드를 사용하여 0~9까지의 손글씨를 분류하는 모델입니다.

In [57]:
# (1) 필요한 모듈 import
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# (2) 데이터 준비
digits = load_digits()
digits_data = digits.data
digits_label = digits.target

# (3) train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(digits_data, 
                                                    digits_label, 
                                                    test_size=0.2, 
                                                    random_state=7)

# (4) 모델 학습 및 예측

# (4-1) Decision Tree
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(random_state=32)
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)

print("- Decision Tree")
print(classification_report(y_test, y_pred))

# (4-2) Random Forest
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(random_state=32)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

print("- Random Forest")
print(classification_report(y_test, y_pred))

# (4-3) SVM
from sklearn import svm

svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

print("- SVM")
print(classification_report(y_test, y_pred))

# (4-4) SGD Classifier
from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier()
sgd_model.fit(X_train, y_train)
y_pred = sgd_model.predict(X_test)

print("- SGD Classifier")
print(classification_report(y_test, y_pred))

# (4-5) Logistic Regression
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
y_pred = logistic_model.predict(X_test)

print("- Logistic Regression")
print(classification_report(y_test, y_pred))

- Decision Tree
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        43
           1       0.81      0.81      0.81        42
           2       0.79      0.82      0.80        40
           3       0.79      0.91      0.85        34
           4       0.83      0.95      0.89        37
           5       0.90      0.96      0.93        28
           6       0.84      0.93      0.88        28
           7       0.96      0.82      0.89        33
           8       0.88      0.65      0.75        43
           9       0.78      0.78      0.78        32

    accuracy                           0.86       360
   macro avg       0.86      0.86      0.86       360
weighted avg       0.86      0.86      0.85       360

- Random Forest
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        43
           1       0.93      1.00      0.97        42
           2       1.00      1.00      1.00   

손글씨 판별기는 단순히 숫자를 분류하는 모델이기 때문에 accuracy가 중요해 보이네요. SVM모델이 0.99로 압도적으로 높은 정확도를 보여주는데, 추측컨데 숫자는 선형분류로 분류하기 쉬운 예제인가 봅니다.

### 와인 분류기
이번에는 load_wine메서드를 사용해서 와인분류기를 만들어 보겠습니다.

In [58]:
# (1) 필요한 모듈 import
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# (2) 데이터 준비
wine = load_wine()
wine_data = wine.data
wine_label = wine.target

# (3) train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(wine_data, 
                                                    wine_label, 
                                                    test_size=0.4, 
                                                    random_state=7)

# (4) 모델 학습 및 예측

# (4-1) Decision Tree
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(random_state=32)
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)

print("- Decision Tree")
print(classification_report(y_test, y_pred))

# (4-2) Random Forest
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(random_state=32)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

print("- Random Forest")
print(classification_report(y_test, y_pred))

# (4-3) SVM
from sklearn import svm

svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

print("- SVM")
print(classification_report(y_test, y_pred))

# (4-4) SGD Classifier
from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier()
sgd_model.fit(X_train, y_train)
y_pred = sgd_model.predict(X_test)

print("- SGD Classifier")
print(classification_report(y_test, y_pred))

# (4-5) Logistic Regression
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
y_pred = logistic_model.predict(X_test)

print("- Logistic Regression")
print(classification_report(y_test, y_pred))

- Decision Tree
              precision    recall  f1-score   support

           0       0.82      0.95      0.88        19
           1       0.93      0.88      0.90        32
           2       1.00      0.95      0.98        21

    accuracy                           0.92        72
   macro avg       0.92      0.92      0.92        72
weighted avg       0.92      0.92      0.92        72

- Random Forest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        32
           2       1.00      1.00      1.00        21

    accuracy                           1.00        72
   macro avg       1.00      1.00      1.00        72
weighted avg       1.00      1.00      1.00        72

- SVM
              precision    recall  f1-score   support

           0       0.86      0.95      0.90        19
           1       0.60      0.94      0.73        32
           2       0.00      0.00    

와인 분류기도 단순한 이미지 분류기이기 때문에 accuracy가 중요해보이네요! 그런데 Random Forest가 1.00으로 가장 높은 점수가 나왔네요. 이 말은 모든 테스트를 다 맞췄다는 말인데, 복잡한 이미지를 분류하는 작업이라 Logistic Regression이 가장 높을 것으로 예상했는데 의외의 결과였습니다.

### 유방암 판별기

마지막으로 load_breast_cancer 메서드를 사용해서 유방암 판별기를 만들어 보겠습니다.



In [59]:
# (1) 필요한 모듈 import
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# (2) 데이터 준비
breast_cancer = load_breast_cancer()
breast_cancer_data = breast_cancer.data
breast_cancer_label = breast_cancer.target

# (3) train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(breast_cancer_data, 
                                                    breast_cancer_label, 
                                                    test_size=0.4, 
                                                    random_state=7)

# (4) 모델 학습 및 예측

# (4-1) Decision Tree
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(random_state=32)
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)

print("- Decision Tree")
print(classification_report(y_test, y_pred))

# (4-2) Random Forest
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(random_state=32)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

print("- Random Forest")
print(classification_report(y_test, y_pred))

# (4-3) SVM
from sklearn import svm

svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

print("- SVM")
print(classification_report(y_test, y_pred))

# (4-4) SGD Classifier
from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier()
sgd_model.fit(X_train, y_train)
y_pred = sgd_model.predict(X_test)

print("- SGD Classifier")
print(classification_report(y_test, y_pred))

# (4-5) Logistic Regression
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
y_pred = logistic_model.predict(X_test)

print("- Logistic Regression")
print(classification_report(y_test, y_pred))

- Decision Tree
              precision    recall  f1-score   support

           0       0.86      0.84      0.85        73
           1       0.92      0.94      0.93       155

    accuracy                           0.90       228
   macro avg       0.89      0.89      0.89       228
weighted avg       0.90      0.90      0.90       228

- Random Forest
              precision    recall  f1-score   support

           0       0.96      0.90      0.93        73
           1       0.96      0.98      0.97       155

    accuracy                           0.96       228
   macro avg       0.96      0.94      0.95       228
weighted avg       0.96      0.96      0.96       228

- SVM
              precision    recall  f1-score   support

           0       0.98      0.78      0.87        73
           1       0.91      0.99      0.95       155

    accuracy                           0.93       228
   macro avg       0.94      0.89      0.91       228
weighted avg       0.93      0.93   

위에서 언급했듯이 암 판결기와 같은 분류기는 Recall값이 중요합니다. 이번에도 Random Forest가 가장 높은 점수가 나왔네요. 

이번에는 여러가지 모델들을 사용해서 비교를 해봤는데요. 집단지성의 힘을 깨달을 수 있는 좋은 시간이었던 것 같습니다!